# Translate FR to EN

In [2]:
import numpy as np
import pandas as pd
print(np.__version__)
print(pd.__version__)

1.19.4
1.0.5


In [3]:
from transformers import MarianMTModel, MarianTokenizer
src_text = [
    "bonjour je voudrais des chips s'il vous plaît",
    "serait-il possible de faire moins de bruit ?",
    "les burgers sont-ils cuits ?",
    "et ça vous fait rire !"
]
src_text

["bonjour je voudrais des chips s'il vous plaît",
 'serait-il possible de faire moins de bruit ?',
 'les burgers sont-ils cuits ?',
 'et ça vous fait rire !']

In [4]:
model_name = 'Helsinki-NLP/opus-mt-fr-en'
marianmt_tokenizer = MarianTokenizer.from_pretrained(model_name)

marianmt_model = MarianMTModel.from_pretrained(model_name)
translated = marianmt_model.generate(**marianmt_tokenizer(src_text, return_tensors="pt", padding=True))

translation = [marianmt_tokenizer.decode(t, skip_special_tokens=True) for t in translated]

translation

[]


["Hello, I'd like some chips, please.",
 'Would it be possible to make less noise?',
 'Are the burgers cooked?',
 'And that makes you laugh!']

# Predict sentiment (for twitter)

In [6]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Preprocess text (username and link placeholders)
def preprocess(text):
    new_text = []
 
 
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

sentiment_tokenizer = AutoTokenizer.from_pretrained(MODEL)
sentiment_tokenizer.save_pretrained(MODEL)

# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]

# PT
sentiment_model = AutoModelForSequenceClassification.from_pretrained(MODEL)
sentiment_model.save_pretrained(MODEL)



OSError: Model name 'cardiffnlp/twitter-roberta-base-sentiment' was not found in tokenizers model name list (roberta-base, roberta-large, roberta-large-mnli, distilroberta-base, roberta-base-openai-detector, roberta-large-openai-detector). We assumed 'cardiffnlp/twitter-roberta-base-sentiment' was a path, a model identifier, or url to a directory containing vocabulary files named ['vocab.json', 'merges.txt'] but couldn't find such vocabulary files at this path or url.

In [7]:
def preprocess_and_predict_sentiment(txt):
    txt = preprocess(txt)
    encoded_input = sentiment_tokenizer(txt, return_tensors='pt')
    output = sentiment_model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return scores

In [36]:
text

'Good night 😊'

In [38]:
text = "Good night 😊"

preprocess_and_predict_sentiment(text).shape


(6, 59514)

In [31]:
[preprocess_and_predict_sentiment(text) for text in translation]

[array([[3.39661754e-04, 1.19656454e-08, 1.47925981e-03, ...,
         1.35539384e-08, 1.37483847e-08, 8.55769122e-06],
        [8.50758166e-04, 4.29434017e-08, 1.79157476e-03, ...,
         4.48998705e-08, 4.54324258e-08, 8.55769122e-06],
        [2.53506820e-04, 1.77586088e-08, 2.04355828e-03, ...,
         2.53600838e-08, 2.65038391e-08, 8.55769122e-06],
        ...,
        [6.96184361e-05, 1.53040016e-08, 6.98564923e-04, ...,
         1.71027015e-08, 1.67914624e-08, 8.55769122e-06],
        [8.71811135e-05, 4.79903983e-09, 9.69734334e-04, ...,
         5.22298871e-09, 5.20541477e-09, 8.55769122e-06],
        [8.13537091e-03, 1.44202525e-08, 2.63848706e-05, ...,
         1.51296273e-08, 1.55429003e-08, 8.55769122e-06]], dtype=float32),
 array([[5.8947215e-03, 4.3190330e-08, 2.3677819e-03, ..., 4.3955236e-08,
         4.4659018e-08, 6.3257619e-05],
        [4.1102016e-04, 9.1906669e-09, 4.8542814e-05, ..., 9.8096100e-09,
         1.0106302e-08, 6.3257619e-05],
        [1.9893670e-04

In [26]:
ranking = np.argsort(scores)[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")


1) positive 0.8466
2) neutral 0.1458
3) negative 0.0076
